### Intent Classification with Rasa NLU and SpaCy 
+ + A Libary for intent recognition and entity extraction based on SpaCy and Sklearn

##### NLP = NLU+NLG+ More
+ NLP = understand,process,interprete everyday human language
+ NLU = unstructured inputs and convert them into a structured form that a machine can understand and act upon

#### Uses
+ Chatbot task
+ NL understanding
+ Intent classification



![alt text](nlu_nlp_explain.png "Title")

#### Installation
+ pip install rasa_nlu
+ python -m rasa_nlu.server &
+ sklearn_crfsuite

#### using spacy as backend
+ pip install rasa_nlu[spacy]
+ python -m spacy download en_core_web_md
+ python -m spacy link en_core_web_md en
  
  = = Dataset = =
+ demo-rasa.json
+ config_spacy.yaml

In [15]:
# Load the Packages
from rasa_nlu.training_data  import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [16]:
# Load Data 
!cat rasa_dataset.json

{
  "rasa_nlu_data": {
    "regex_features": [
      {
        "name": "zipcode",
        "pattern": "[0-9]{5}"
      },
      {
        "name": "greet",
        "pattern": "hey[^\\s]*"
      }
    ],
    "entity_synonyms": [
      {
        "value": "chinese",
        "synonyms": ["Chinese", "Chines", "chines"]
      },
      {
        "value": "vegetarian",
        "synonyms": ["veggie", "vegg"]
      }
    ],
    "common_examples": [
      {
        "text": "hey", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "howdy", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hey there",
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hello", 
        "intent": "greet", 
        "entities": []
      }, 
      {
        "text": "hi", 
        "intent": "greet", 
        "entities": []
      },
      {
        "text": "good morning",
        "intent": "greet",
        "entities": 

In [17]:
# Loading DataSet
train_data = load_data('rasa_dataset.json')

In [18]:
# Config Backend using Sklearn and Spacy
trainer = Trainer(config.load("config_spacy.yaml"))

#### Content on Config
    language: "en"
    pipeline: "spacy_sklearn"

    =======================

    language: "en"

    pipeline:
    - name: "nlp_spacy"
    - name: "tokenizer_spacy"
    - name: "intent_entity_featurizer_regex"
    - name: "intent_featurizer_spacy"
    - name: "ner_crf"
    - name: "ner_synonyms"
    - name: "intent_classifier_sklearn"

In [19]:
# Training Data
trainer.train(train_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished


In [20]:
# Returns the directory the model is stored in (Creat a folder to store model in)
model_directory = trainer.persist('/projects/')

#### Entity Extraction With SpaCy

In [22]:
import spacy
nlp = spacy.load('en')

In [23]:
docx = nlp(u"I am looking for an Italian Restaurant where I can eat")

In [24]:
for word in docx.ents:
    print("value",word.text,"entity",word.label_,"start",word.start_char,"end",word.end_char)

value Italian entity NORP start 20 end 27


#### Making Predictions With Model
+ Interpreter.load().parse()

In [21]:
from rasa_nlu.model import Metadata, Interpreter

In [25]:
# where `model_directory points to the folder the model is persisted in
interpreter = Interpreter.load(model_directory)

In [26]:
# Prediction of Intent
interpreter.parse(u"I am looking for an Italian Restaurant where I can eat")

{'intent': {'name': 'restaurant_search', 'confidence': 0.7455215289019911},
 'entities': [{'start': 20,
   'end': 27,
   'value': 'italian',
   'entity': 'cuisine',
   'confidence': 0.6636828413532201,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.7455215289019911},
  {'name': 'affirm', 'confidence': 0.15019642212447237},
  {'name': 'greet', 'confidence': 0.058736824115844515},
  {'name': 'goodbye', 'confidence': 0.045545224857692024}],
 'text': 'I am looking for an Italian Restaurant where I can eat'}

In [27]:
interpreter.parse(u"I want an African Spot to eat")

{'intent': {'name': 'restaurant_search', 'confidence': 0.6874972430877329},
 'entities': [{'start': 10,
   'end': 17,
   'value': 'african',
   'entity': 'cuisine',
   'confidence': 0.6470976966769572,
   'extractor': 'ner_crf'}],
 'intent_ranking': [{'name': 'restaurant_search',
   'confidence': 0.6874972430877329},
  {'name': 'goodbye', 'confidence': 0.12400667696797882},
  {'name': 'affirm', 'confidence': 0.11357435021080386},
  {'name': 'greet', 'confidence': 0.07492172973348454}],
 'text': 'I want an African Spot to eat'}

In [28]:
interpreter.parse(u"Good morning World")

{'intent': {'name': 'greet', 'confidence': 0.44328419685532383},
 'entities': [],
 'intent_ranking': [{'name': 'greet', 'confidence': 0.44328419685532383},
  {'name': 'goodbye', 'confidence': 0.31245698090344237},
  {'name': 'affirm', 'confidence': 0.1257434275305043},
  {'name': 'restaurant_search', 'confidence': 0.11851539471072912}],
 'text': 'Good morning World'}

In [ ]:
### Credits Rasa_nlu
#### By Jesse JCharis
#### Jesus Saves @ JCharisTec